# Codigo de Clase + Test Shorts

## Full-optimizado


In [ ]:
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
import optuna

# import emyrical ← No funciona en Python 3.13 :(

In [ ]:
# --- 1. CONFIGURACIÓN Y CLASES DE DATOS ---

# Gonfig Graficos
sns.set_theme(style="whitegrid")

@dataclass
class Operation:
    """Clase para almacenar la información de una operación de trading."""
    open_time: pd.Timestamp
    open_price: float
    n_shares: int
    type: str
    stop_loss: float
    take_profit: float
    status: str = 'OPEN'
    close_time: pd.Timestamp = None
    close_price: float = None
    pnl: float = 0.0

In [ ]:
# --- 2. FUNCIONES DE CÁLCULO DE INDICADORES Y SEÑALES ---

def calculate_indicators(df: pd.DataFrame, params: dict) -> pd.DataFrame:
    """Calcula y añade los indicadores técnicos al DataFrame."""
    df_copy = df.copy()
    df_copy['rsi'] = ta.momentum.RSIIndicator(df_copy['Close'], window=params['rsi_window']).rsi()
    bollinger = ta.volatility.BollingerBands(df_copy['Close'], window=params['bb_window'], window_dev=2)
    df_copy['bb_high'] = bollinger.bollinger_hband()
    df_copy['bb_low'] = bollinger.bollinger_lband()
    stochastic = ta.momentum.StochasticOscillator(
        df_copy['High'], df_copy['Low'], df_copy['Close'],
        window=params['stoch_window'], smooth_window=params['stoch_smooth_k']
    )
    df_copy['stoch_k'] = stochastic.stoch()
    return df_copy.dropna()

def generate_signals(df: pd.DataFrame, params: dict) -> pd.DataFrame:
    """Genera señales de compra y venta basadas en el consenso de los indicadores."""
    df_copy = df.copy()
    rsi_buy = df_copy['rsi'] < params['rsi_buy_th']
    rsi_sell = df_copy['rsi'] > params['rsi_sell_th']
    bb_buy = df_copy['Close'] < df_copy['bb_low']
    bb_sell = df_copy['Close'] > df_copy['bb_high']
    stoch_buy = df_copy['stoch_k'] < params['stoch_buy_th']
    stoch_sell = df_copy['stoch_k'] > params['stoch_sell_th']
    df_copy['buy_signal'] = (rsi_buy.astype(int) + bb_buy.astype(int) + stoch_buy.astype(int)) >= 2
    df_copy['sell_signal'] = (rsi_sell.astype(int) + bb_sell.astype(int) + stoch_sell.astype(int)) >= 2
    return df_copy

In [ ]:
# --- 3. LÓGICA DEL BACKTESTING ---

def run_backtest(df: pd.DataFrame, initial_cash: float, n_shares: int, commission: float, sl_pct: float, tp_pct: float):
    """Ejecuta la simulación de trading (backtest) iterando a través de los datos."""
    cash = initial_cash
    active_positions: list[Operation] = []
    closed_trades_log: list[Operation] = []
    portfolio_history = []

    for timestamp, row in df.iterrows():
        current_price = row['Close']
        for position in active_positions[:]:
            if position.type == 'LONG' and (current_price >= position.take_profit or current_price <= position.stop_loss):
                cash += current_price * position.n_shares * (1 - commission)
                position.status = 'CLOSED'
                position.close_time = timestamp
                position.close_price = current_price
                position.pnl = ((position.close_price - position.open_price) * position.n_shares -
                                (position.open_price + position.close_price) * position.n_shares * commission)
                closed_trades_log.append(position)
                active_positions.remove(position)
            elif position.type == 'SHORT' and (current_price <= position.take_profit or current_price >= position.stop_loss):
                cash -= current_price * position.n_shares * (1 + commission)
                position.status = 'CLOSED'
                position.close_time = timestamp
                position.close_price = current_price
                position.pnl = ((position.open_price - position.close_price) * position.n_shares -
                                (position.open_price + position.close_price) * position.n_shares * commission)
                closed_trades_log.append(position)
                active_positions.remove(position)

        cost_of_long = current_price * n_shares * (1 + commission)
        if row['buy_signal'] and cash >= cost_of_long:
            cash -= cost_of_long
            active_positions.append(Operation(
                open_time=timestamp, open_price=current_price, n_shares=n_shares, type='LONG',
                stop_loss=current_price * (1 - sl_pct), take_profit=current_price * (1 + tp_pct)
            ))
        elif row['sell_signal']:
            cash += current_price * n_shares * (1 - commission)
            active_positions.append(Operation(
                open_time=timestamp, open_price=current_price, n_shares=n_shares, type='SHORT',
                stop_loss=current_price * (1 + sl_pct), take_profit=current_price * (1 - tp_pct)
            ))

        long_value = sum(p.n_shares * current_price for p in active_positions if p.type == 'LONG')
        short_liability = sum(p.n_shares * current_price for p in active_positions if p.type == 'SHORT')
        portfolio_value = cash + long_value - short_liability
        portfolio_history.append({'timestamp': timestamp, 'value': portfolio_value})

    last_price = df['Close'].iloc[-1]
    for pos in active_positions:
        if pos.type == 'LONG': cash += last_price * pos.n_shares * (1 - commission)
        elif pos.type == 'SHORT': cash -= last_price * pos.n_shares * (1 + commission)
    if portfolio_history:
        portfolio_history[-1]['value'] = cash

    return pd.DataFrame(portfolio_history).set_index('timestamp')['value'], closed_trades_log

In [ ]:
# --- 4. MÉTRICAS DE DESEMPEÑO Y VISUALIZACIÓN ---

def calculate_performance_metrics(
    portfolio_values: pd.Series,
    trades_log: list[Operation],
    time_frame_minutes: int
) -> dict:
    """Calcula las métricas clave de desempeño de la estrategia de forma manual."""
    if portfolio_values.empty or len(trades_log) == 0:
        return {
            'Calmar Ratio': 0.0, 'Sharpe Ratio': 0.0, 'Sortino Ratio': 0.0,
            'Max Drawdown': 0.0, 'Win Rate': 0.0, 'Total Trades': 0,
            'Annualized Return': 0.0
        }

    # 1. Calcular retornos por cada vela
    returns = portfolio_values.pct_change().dropna()
    if returns.empty:
        return {
            'Calmar Ratio': 0.0, 'Sharpe Ratio': 0.0, 'Sortino Ratio': 0.0,
            'Max Drawdown': 0.0, 'Win Rate': 0.0, 'Total Trades': len(trades_log),
            'Annualized Return': 0.0
        }

    # 2. Factor de anualización para mercado crypto (24/7)
    bars_per_year = (24 * 60 / time_frame_minutes) * 365

    # 3. Retorno anualizado
    mean_return_per_bar = returns.mean()
    annualized_return = mean_return_per_bar * bars_per_year

    # 4. Sharpe Ratio
    std_dev_per_bar = returns.std()
    annualized_std_dev = std_dev_per_bar * np.sqrt(bars_per_year)
    sharpe_ratio = annualized_return / annualized_std_dev if annualized_std_dev != 0 else 0.0

    # 5. Sortino Ratio
    downside_returns = returns[returns < 0]
    downside_std_dev_per_bar = downside_returns.std()
    annualized_downside_risk = downside_std_dev_per_bar * np.sqrt(bars_per_year)
    sortino_ratio = annualized_return / annualized_downside_risk if annualized_downside_risk != 0 else 0.0

    # 6. Max Drawdown
    cumulative_max = portfolio_values.cummax()
    drawdown = (cumulative_max - portfolio_values) / cumulative_max
    max_drawdown = drawdown.max()

    # 7. Calmar Ratio
    calmar_ratio = annualized_return / max_drawdown if max_drawdown != 0 else 0.0

    # 8. Win Rate
    win_rate = sum(1 for trade in trades_log if trade.pnl > 0) / len(trades_log)

    metrics = {
        'Annualized Return': annualized_return,
        'Calmar Ratio': calmar_ratio,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Max Drawdown': max_drawdown,
        'Win Rate': win_rate,
        'Total Trades': len(trades_log)
    }
    return {key: round(value, 4) for key, value in metrics.items()}

def plot_portfolio(portfolio_values: pd.Series, title: str):
    """Grafica la evolución del valor del portafolio."""
    plt.figure(figsize=(15, 7))
    portfolio_values.plot(title=title, ylabel='Valor del Portafolio (USD)', xlabel='Fecha')
    plt.show()

In [ ]:
# --- 5. FUNCIÓN OBJETIVO PARA OPTIMIZACIÓN ---

def objective(trial: optuna.trial.Trial, df_train: pd.DataFrame, time_frame_minutes: int) -> float:
    """Función que Optuna intentará maximizar."""
    # Parámetros de los indicadores
    params = {
        'rsi_window': trial.suggest_int('rsi_window', 10, 50),
        'bb_window': trial.suggest_int('bb_window', 10, 50),
        'stoch_window': trial.suggest_int('stoch_window', 10, 50),
        'stoch_smooth_k': trial.suggest_int('stoch_smooth_k', 3, 10),
        'rsi_buy_th': trial.suggest_int('rsi_buy_th', 20, 40),
        'rsi_sell_th': trial.suggest_int('rsi_sell_th', 60, 80),
        'stoch_buy_th': trial.suggest_int('stoch_buy_th', 10, 30),
        'stoch_sell_th': trial.suggest_int('stoch_sell_th', 70, 90),
    }

    sl_pct = trial.suggest_float('stop_loss', 0.01, 0.12)
    tp_pct = trial.suggest_float('take_profit', 0.01, 0.15)
    n_shares = trial.suggest_int('n_shares', 1, 10) # Nota: Empezamos en 1, 0 no tiene sentido.

    # Constantes del proyecto
    INITIAL_CASH, COMMISSION = 10_000_000, 0.00125

    # --- Se usan los nuevos parámetros en el backtest ---
    df_indicators = calculate_indicators(df_train, params)
    df_signals = generate_signals(df_indicators, params)
    portfolio_values, trades_log = run_backtest(df_signals, INITIAL_CASH, n_shares, COMMISSION, sl_pct, tp_pct)

    if portfolio_values.empty or len(trades_log) < 10:
        return -1.0

    metrics = calculate_performance_metrics(portfolio_values, trades_log, time_frame_minutes)
    return metrics['Calmar Ratio']

In [ ]:
# --- 6. FUNCIÓN PRINCIPAL ---

def main():
    """Función principal que ejecuta todo el proceso."""
    # Constantes y configuración
    DATA_FILE = 'Binance_BTCUSDT_1h.csv'
    TIME_FRAME_MINUTES = 60

    try:
        data = pd.read_csv(DATA_FILE, skiprows=1)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{DATA_FILE}'.")
        return

    data['Date'] = pd.to_datetime(data['Date'], format='mixed')
    data = data.set_index('Date').sort_index()

    train_size = int(len(data) * 0.6)
    test_size = int(len(data) * 0.2)
    df_train, df_test = data.iloc[:train_size], data.iloc[train_size:train_size + test_size]

    print(f"Datos cargados: {len(data)} velas. Marco de tiempo: {TIME_FRAME_MINUTES} min.")

    print("\nIniciando optimización de parámetros...")
    study = optuna.create_study(direction='maximize')
    study.optimize(
        lambda trial: objective(trial, df_train, TIME_FRAME_MINUTES),
        n_trials=100, # Aumentar n_trials puede dar mejores resultados
        show_progress_bar=True
    )

    print(f"\nOptimización completada. Mejor Calmar Ratio: {study.best_value:.4f}")
    best_params = study.best_params
    print(f"Mejores parámetros: {best_params}")

    print("\nEvaluando la estrategia en el set de prueba...")

    INITIAL_CASH = 1_000_000
    COMMISSION = 0.00125

    # Los parámetros de los indicadores ya están en best_params
    df_test_indicators = calculate_indicators(df_test, best_params)
    df_test_signals = generate_signals(df_test_indicators, best_params)

    # Se usan los parámetros de trading óptimos
    portfolio_values, trades_log = run_backtest(
        df_test_signals,
        INITIAL_CASH,
        n_shares=best_params['n_shares'],
        commission=COMMISSION,
        sl_pct=best_params['stop_loss'],
        tp_pct=best_params['take_profit']
    )

    if not portfolio_values.empty:
        final_metrics = calculate_performance_metrics(portfolio_values, trades_log, TIME_FRAME_MINUTES)
        final_value = portfolio_values.iloc[-1]
        print(f"- Valor Final del Portafolio:   ${final_value:,.2f} USD")
        print("\n--- Métricas de Desempeño Finales (Set de Prueba) ---")
        for key, value in final_metrics.items():
            print(f"- {key}: {value}")
        plot_portfolio(portfolio_values, "Evolución del Portafolio (Set de Prueba)")
    else:
        print("\nNo se realizaron operaciones en el set de prueba.")